In [38]:
import torch
import torchvision
from torchvision import transforms,datasets

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F
import pdb

In [75]:
data = pd.read_csv('Joint_Dataset.csv')

In [76]:
data.head()

,left_shoulder_X,left_shoulder_Y,left_shoulder_Z,left_elbow_X,left_elbow_Y,left_elbow_Z,left_wrist_X,left_wrist_Y,left_wrist_Z,left_hand_X,...,right_knee_X,right_knee_Y,right_knee_Z,right_ankle_X,right_ankle_Y,right_ankle_Z,right_foot_X,right_foot_Y,right_foot_Z,label
0,120.0,80.0,-269.0,114.0,108.0,-267.0,116.0,139.0,-255.0,115.0,...,153.0,182.0,-276.0,154.0,225.0,-285.0,153.0,234.0,-243.0,1
1,121.0,81.0,-268.0,114.0,109.0,-266.0,115.0,140.0,-255.0,117.0,...,153.0,182.0,-276.0,154.0,226.0,-284.0,154.0,234.0,-242.0,1
2,120.0,81.0,-269.0,114.0,109.0,-266.0,115.0,140.0,-257.0,117.0,...,153.0,182.0,-276.0,155.0,226.0,-283.0,155.0,234.0,-241.0,1
3,120.0,81.0,-269.0,113.0,109.0,-266.0,115.0,140.0,-255.0,116.0,...,153.0,182.0,-275.0,154.0,225.0,-286.0,154.0,234.0,-244.0,1
4,120.0,81.0,-269.0,114.0,109.0,-267.0,115.0,140.0,-255.0,117.0,...,153.0,182.0,-275.0,154.0,225.0,-285.0,154.0,234.0,-243.0,1


In [77]:
X = data.drop('label',axis=1)
Y = data['label']
X.shape,Y.shape

((22797, 60), (22797,))

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [79]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
#y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
y_th_train = torch.from_numpy(y_train.to_numpy()).type(torch.LongTensor)
y_th_train = y_th_train-1

X_test = torch.from_numpy(X_test.to_numpy()).float()
#y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())
y_th_test = torch.from_numpy(y_test.to_numpy()).type(torch.LongTensor)
y_th_test=y_th_test-1

#rint(X_train.shape, y_train.shape)
#rint(X_test.shape, y_test.shape)

In [81]:
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 40)
        self.fc2 = nn.Linear(40, 40)
        self.fc3 = nn.Linear(40, 40)
        self.fc4 = nn.Linear(40,40)
        self.fc5 = nn.Linear(40,20)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x,dim=1)

In [82]:
net = Net(X_train.shape[1])
#ann_viz(net, view=False)

In [83]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [84]:
def calculate_accuracy(y_true, y_pred):
    predicted = y_pred.ge(.5).view(-1)
    return (y_true == predicted).sum().float() / len(y_true)

In [85]:
def round_tensor(t, decimal_places=3):
    return round(t.item(), decimal_places)

In [86]:

for epoch in range(1000):
    y_pred = net(X_train)
    #pdb.set_trace()
    #optimizer.zero_grad()
    train_loss = criterion(y_pred,y_th_train)
    if epoch % 100 == 0:
        #pdb.set_trace()
        y_pred = y_pred.argmax(dim=1)
        train_acc = calculate_accuracy(y_th_train, y_pred)
        #pdb.set_trace()
        y_test_pred = net(X_test)
        test_loss = criterion(y_test_pred, y_th_test)
        y_test_pred = y_test_pred.argmax(dim=1)
        test_acc = calculate_accuracy(y_th_test, y_test_pred)
        print(f'''epoch {epoch} Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)*100} Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)*100} ''')
        
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

epoch 0 Train set - loss: 19650.602, accuracy: 5.2 Test  set - loss: 17926.424, accuracy: 4.8 
epoch 100 Train set - loss: 140.026, accuracy: 6.1 Test  set - loss: 179.069, accuracy: 5.2 
epoch 200 Train set - loss: 165.377, accuracy: 7.3 Test  set - loss: 147.435, accuracy: 6.4 
epoch 300 Train set - loss: 148.573, accuracy: 7.3 Test  set - loss: 110.559, accuracy: 6.5 
epoch 400 Train set - loss: 89.973, accuracy: 7.3999999999999995 Test  set - loss: 77.384, accuracy: 6.5 
epoch 500 Train set - loss: 52.287, accuracy: 7.6 Test  set - loss: 62.641, accuracy: 6.4 
epoch 600 Train set - loss: 68.02, accuracy: 7.3999999999999995 Test  set - loss: 81.178, accuracy: 6.5 
epoch 700 Train set - loss: 50.873, accuracy: 7.3999999999999995 Test  set - loss: 39.847, accuracy: 6.5 
epoch 800 Train set - loss: 14.206, accuracy: 7.3999999999999995 Test  set - loss: 12.841, accuracy: 6.4 
epoch 900 Train set - loss: 34.583, accuracy: 7.5 Test  set - loss: 26.926, accuracy: 6.5 
